# Deployment Pipeline

**SageMaker Studio Kernel**: Data Science

In this exercise you will do:
 - Run the end to end workflow described in the previous exercise for deploying an Amazon SageMaker Endpoint

***

## Part 1/3 - Setup
Here we'll import some libraries and define some variables.

In [ ]:
import boto3
import json
import logging
import os
import sagemaker
from sagemaker import get_execution_role
import sys

In [ ]:
sys.path.insert(0, os.path.abspath('./../mlpipelines'))

In [ ]:
from deployment.pipeline import get_pipeline

In [ ]:
s3_client = boto3.client('s3')
sm_client = boto3.client('sagemaker')

In [ ]:
logging.basicConfig(level=logging.INFO)
LOGGER = logging.getLogger(__name__)

***

## Part 2/2 - Run the Pipeline 

### Explore Pipeline definition

In [ ]:
!pygmentize ./../mlpipelines/deployment/pipeline.py

### Pipeline Parameters

In [ ]:
region = boto3.session.Session().region_name
role_name = "mlops-sagemaker-execution-role"
role = "arn:aws:iam::{}:role/{}".format(boto3.client('sts').get_caller_identity().get('Account'), role_name)

env = "dev"

kms_account_id = boto3.client('sts').get_caller_identity().get('Account')

kms_alias = "ml-kms"

bucket_artifacts = ""
bucket_inference = ""

inference_artifact_path = "artifact/inference"
inference_artifact_name = "sourcedir.tar.gz"
inference_instance_count = 1
inference_instance_type = "ml.m5.xlarge"

model_package_group = "ml-end-to-end-group"

monitoring_output_path = "data/monitoring/captured"

training_framework_version = 2.4

### Compress source code for installing additional python modules

In [ ]:
! ./../algorithms/buildspec.sh deployment $bucket_artifacts

***

### Deployment pipeline

#### Get pipeline definition

In [ ]:
pipeline = get_pipeline(
    region,
    env,
    kms_account_id,
    kms_alias,
    bucket_artifacts,
    bucket_inference,
    inference_artifact_path,
    inference_artifact_name,
    inference_instance_count,
    inference_instance_type,
    model_package_group,
    monitoring_output_path,
    training_framework_version,
    role
)